In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install netCDF4
!pip install xarray

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 48.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 79.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### Imports

In [ ]:
import numpy as np
import netCDF4 as nc
import scipy.io as sio
import xarray as xr
import pickle
from scipy.stats import pearsonr

In [ ]:
import tensorflow
import keras.backend as K
import netCDF4
import numpy as np
from keras.layers import Input, Convolution2D, Convolution1D, MaxPooling2D, Dense, Dropout, \
                          Flatten, concatenate, Activation, Reshape, \
                          UpSampling2D,ZeroPadding2D
import keras
from keras.callbacks import History
history = History()

import keras
from keras.layers import Conv2D, Conv2DTranspose, Cropping2D, Concatenate, ZeroPadding2D
from keras.models import load_model
from keras.layers import Input
from keras.models import Model
from keras.layers import Activation
from keras.layers import MaxPool2D
from keras.layers import Flatten
from keras.layers import Dense

# from utils import get_initial_weights
# from layers import BilinearInterpolation

__version__ = 0.1 

In [ ]:
#Getting weights for spatial transofmer module's affine transformation theta
def get_initial_weights(output_size):
    b = np.zeros((2, 3), dtype='float32')
    b[0, 0] = 1
    b[1, 1] = 1
    W = np.zeros((output_size, 6), dtype='float32')
    weights = [W, b.flatten()]
    return weights

In [ ]:
import keras.backend as K
#from keras.engine.topology import Layer
from tensorflow.python.keras.layers import Layer

if K.backend() == 'tensorflow':
    import tensorflow as tf

    def K_meshgrid(x, y):
        return tf.meshgrid(x, y)

    def K_linspace(start, stop, num):
        return tf.linspace(start, stop, num)

else:
    raise Exception("Only 'tensorflow' is supported as backend")


class BilinearInterpolation(Layer):
    """Performs bilinear interpolation as a keras layer
    References
    ----------
    [1]  Spatial Transformer Networks, Max Jaderberg, et al.
    [2]  https://github.com/skaae/transformer_network
    [3]  https://github.com/EderSantana/seya
    """

    def __init__(self, output_size, **kwargs):
        self.output_size = output_size
        super(BilinearInterpolation, self).__init__(**kwargs)

    def get_config(self):
        return {
            'output_size': self.output_size,
        }

    def compute_output_shape(self, input_shapes):
        height, width = self.output_size
        num_channels = input_shapes[0][-1]
        return (None, height, width, num_channels)

    def call(self, tensors, mask=None):
        X, transformation = tensors
        output = self._transform(X, transformation, self.output_size)
        return output

    def _interpolate(self, image, sampled_grids, output_size):

        batch_size = K.shape(image)[0]
        height = K.shape(image)[1]
        width = K.shape(image)[2]
        num_channels = K.shape(image)[3]

        x = K.cast(K.flatten(sampled_grids[:, 0:1, :]), dtype='float32')
        y = K.cast(K.flatten(sampled_grids[:, 1:2, :]), dtype='float32')

        x = .5 * (x + 1.0) * K.cast(width, dtype='float32')
        y = .5 * (y + 1.0) * K.cast(height, dtype='float32')

        x0 = K.cast(x, 'int32')
        x1 = x0 + 1
        y0 = K.cast(y, 'int32')
        y1 = y0 + 1

        max_x = int(K.int_shape(image)[2] - 1)
        max_y = int(K.int_shape(image)[1] - 1)

        x0 = K.clip(x0, 0, max_x)
        x1 = K.clip(x1, 0, max_x)
        y0 = K.clip(y0, 0, max_y)
        y1 = K.clip(y1, 0, max_y)

        pixels_batch = K.arange(0, batch_size) * (height * width)
        pixels_batch = K.expand_dims(pixels_batch, axis=-1)
        flat_output_size = output_size[0] * output_size[1]
        base = K.repeat_elements(pixels_batch, flat_output_size, axis=1)
        base = K.flatten(base)

        # base_y0 = base + (y0 * width)
        base_y0 = y0 * width
        base_y0 = base + base_y0
        # base_y1 = base + (y1 * width)
        base_y1 = y1 * width
        base_y1 = base_y1 + base

        indices_a = base_y0 + x0
        indices_b = base_y1 + x0
        indices_c = base_y0 + x1
        indices_d = base_y1 + x1

        flat_image = K.reshape(image, shape=(-1, num_channels))
        flat_image = K.cast(flat_image, dtype='float32')
        pixel_values_a = K.gather(flat_image, indices_a)
        pixel_values_b = K.gather(flat_image, indices_b)
        pixel_values_c = K.gather(flat_image, indices_c)
        pixel_values_d = K.gather(flat_image, indices_d)

        x0 = K.cast(x0, 'float32')
        x1 = K.cast(x1, 'float32')
        y0 = K.cast(y0, 'float32')
        y1 = K.cast(y1, 'float32')

        area_a = K.expand_dims(((x1 - x) * (y1 - y)), 1)
        area_b = K.expand_dims(((x1 - x) * (y - y0)), 1)
        area_c = K.expand_dims(((x - x0) * (y1 - y)), 1)
        area_d = K.expand_dims(((x - x0) * (y - y0)), 1)

        values_a = area_a * pixel_values_a
        values_b = area_b * pixel_values_b
        values_c = area_c * pixel_values_c
        values_d = area_d * pixel_values_d
        return values_a + values_b + values_c + values_d

    def _make_regular_grids(self, batch_size, height, width):
        # making a single regular grid
        x_linspace = K_linspace(-1., 1., width)
        y_linspace = K_linspace(-1., 1., height)
        x_coordinates, y_coordinates = K_meshgrid(x_linspace, y_linspace)
        x_coordinates = K.flatten(x_coordinates)
        y_coordinates = K.flatten(y_coordinates)
        ones = K.ones_like(x_coordinates)
        grid = K.concatenate([x_coordinates, y_coordinates, ones], 0)

        # repeating grids for each batch
        grid = K.flatten(grid)
        grids = K.tile(grid, K.stack([batch_size]))
        return K.reshape(grids, (batch_size, 3, height * width))

    def _transform(self, X, affine_transformation, output_size):
        batch_size, num_channels = K.shape(X)[0], K.shape(X)[3]
        transformations = K.reshape(affine_transformation,
                                    shape=(batch_size, 2, 3))
        # transformations = K.cast(affine_transformation[:, 0:2, :], 'float32')
        regular_grids = self._make_regular_grids(batch_size, *output_size)
        sampled_grids = K.batch_dot(transformations, regular_grids)
        interpolated_image = self._interpolate(X, sampled_grids, output_size)
        new_shape = (batch_size, output_size[0], output_size[1], num_channels)
        interpolated_image = K.reshape(interpolated_image, new_shape)
        return interpolated_image


### Load Data

This part of the code takes the hourly Z500 data and computes the mean and standard deviation for normalization 

It takes each netCDF file as one entry to a file list to facilitate memory optimization 

In this [paper](https://gmd.copernicus.org/preprints/gmd-2021-71/) year 1979-2017 is taken together, where 1979-2016 is used as training and 2017 as validation. 

In this example, we have changed the validation set slightly. It would not affect the performance. 



In [ ]:
# Getting back the objects:
with open('/content/drive/MyDrive/DDWP/z500_mean_sdev.pkl', 'rb') as f:  # Python 3: open(..., 'rb')
    M, sdev = pickle.load(f)

M, sdev

(54110.977, 3355.03)

In [ ]:
fileList_train = []
fileList_test=[]
for i in range (1979,2018):
    fileList_train.append (f'/content/drive/MyDrive/DDWP/geopotential_z500/geopotential_500hPa_' + str(i)+'_5.625deg.nc')

for j in range (2018, 2019):
    fileList_test.append (f'/content/drive/MyDrive/DDWP/geopotential_z500/geopotential_500hPa_' + str(j)+'_5.625deg.nc')

### Define and compile model and loss function

We have defined the model as "stn" where "stn" includes an U-NET with a spatial transformer module that includes an affine transformation and Biliniear interpolation

In [ ]:
## border_mode --> padding

def stn(input_shape=(32, 64, 1), sampling_size=(8, 16), num_classes=10):
    inputs = Input(shape=input_shape)
    conv1 = Conv2D(32, (5, 5), activation='relu', padding='same')(inputs)
    conv1 = Conv2D(32, (5, 5), activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D((2,2), padding='same')(conv1)

    conv2 = Conv2D(32, (5, 5), activation='relu', padding='same')(pool1)
    conv2 = Conv2D(32, (5, 5), activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D((2,2), padding='same')(conv2)

    conv3 = Conv2D(32, (5, 5), activation='relu', padding='same')(pool2)
    conv3 = Conv2D(32, (5, 5), activation='relu', padding='same')(conv3)


    conv5 = Conv2D(32, (5, 5), activation='relu', padding='same')(conv3)
    conv5 = Conv2D(32, (5, 5), activation='relu', padding='same')(conv5)
    
    locnet = Flatten()(conv5)
    locnet = Dense(500)(locnet)
    locnet = Activation('relu')(locnet)
    locnet = Dense(200)(locnet)
    locnet = Activation('relu')(locnet)
    locnet = Dense(100)(locnet)
    locnet = Activation('relu')(locnet)
    locnet = Dense(50)(locnet)
    locnet = Activation('relu')(locnet)
    weights = get_initial_weights(50)
    locnet = Dense(6, weights=weights)(locnet)
    x = BilinearInterpolation(sampling_size)([inputs, locnet])


    up6 = keras.layers.Concatenate(axis=-1)([Conv2D(32, (2, 2), activation='relu', padding='same')(UpSampling2D(size=(2, 2))(x)), conv2])
    conv6 = Conv2D(32, (5, 5), activation='relu', padding='same')(up6)
    conv6 = Conv2D(32, (5, 5), activation='relu', padding='same')(conv6)

    up7 = keras.layers.Concatenate(axis=-1)([Conv2D(32, (2, 2), activation='relu', padding='same')(UpSampling2D(size=(2, 2))(conv6)), conv1])
    conv7 = Conv2D(32, (5, 5), activation='relu', padding='same')(up7)
    conv7 = Conv2D(32, (5, 5), activation='relu', padding='same')(conv7)

    conv10 = Conv2D(1, (5, 5), activation='linear',padding='same')(conv7)
    model = Model(inputs=inputs, outputs=conv10)
    return model

In [ ]:
def unet_baseline(input_shape=(32, 64, 1), sampling_size=(8, 16), num_classes=10):
    inputs = Input(shape=input_shape)
    conv1 = Conv2D(32, (5, 5), activation='relu', padding='same')(inputs)
    conv1 = Conv2D(32, (5, 5), activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(32, (5, 5), activation='relu', padding='same')(pool1)
    conv2 = Conv2D(32, (5, 5), activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(32, (5, 5), activation='relu', padding='same')(pool2)
    conv3 = Conv2D(32, (5, 5), activation='relu', padding='same')(conv3)

    conv5 = Conv2D(32, (5, 5), activation='relu', padding='same')(conv3)
    x = Conv2D(32, (5, 5), activation='relu', padding='same')(conv5)

    up6 = keras.layers.Concatenate(axis=-1)([Conv2D(32, (2, 2), activation='relu', padding='same')(UpSampling2D(size=(2, 2))(x)), conv2])
    conv6 = Conv2D(32, (5, 5), activation='relu', padding='same')(up6)
    conv6 = Conv2D(32, (5, 5), activation='relu', padding='same')(conv6)

    up7 = keras.layers.Concatenate(axis=-1)([Conv2D(32, (2, 2),activation='relu', padding='same')(UpSampling2D(size=(2, 2))(conv6)), conv1])
    conv7 = Conv2D(32, (5, 5), activation='relu', padding='same')(up7)
    conv7 = Conv2D(32, (5, 5), activation='relu', padding='same')(conv7)

    conv10 = Conv2D(1, (5, 5), activation='linear', padding='same')(conv7)

    model = Model(inputs=inputs, outputs=conv10)

    return model

In [ ]:
def unet_old(input_shape=(32, 64, 1), sampling_size=(8, 16), num_classes=10):
  
    inputs = Input(shape=(32, 64, 1))

    conv1 = Conv2D(32, 3, activation='relu', padding='same')(inputs)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(32, 3, activation='relu', padding='same')(pool1)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(32, 3, activation='relu', padding='same')(pool2)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    locnet = Flatten()(pool3)
    locnet = Dense(500)(locnet)
    locnet = Activation('relu')(locnet)
    locnet = Dense(250)(locnet)
    locnet = Activation('relu')(locnet)
    locnet = Dense(50)(locnet)
    locnet = Activation('relu')(locnet)
    weights = get_initial_weights(50)
    locnet = Dense(6, weights=weights)(locnet)
    x = BilinearInterpolation((8, 16))([inputs, locnet])

    conv4 = Conv2D(32, (5, 5), activation='relu', padding='same')(x)
    up4 = UpSampling2D(size=(2, 2))(conv4)

    conv5 = Conv2D(32, (5, 5), activation='relu', padding='same')(up4)
    up5 = UpSampling2D(size=(2, 2))(conv5)

    conv6 = Conv2D(1, (5, 5), activation='relu', padding='same')(up5)

    model = Model(inputs=inputs, outputs=conv6)

    return model

In [ ]:
def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true))) 

In [ ]:
# # N is length of training data (number of days)
# N = 10
# gamma = 0.1
# def L1_Loss(y_true, y_pred):
#   loss = (1-gamma)*(y_pred - y_true)**2 + (gamma*y_pred)
#   return loss/N

In [ ]:
# N = 10
# gamma = 0.1
# def L1_Loss(y_true, y_pred):
#   loss = (1-gamma)*(tf.math.reduce_sum((y_pred - y_true)**2)) + (gamma*tf.math.reduce_sum(y_pred))
#   return loss/N

In [ ]:
gamma = 0.01
def L1_Loss(y_true, y_pred):
  term1 = (1 - gamma) * tf.reduce_mean(tf.square(y_pred - y_true))
  term2 = gamma * tf.reduce_mean(y_pred)
  L1 = term1 + term2

  return L1

In [ ]:
N = 10
gamma = 0.25
def L2_Loss(y_true,y_pred):
    diff_pred = y_pred[1:] - y_pred[:-1]
    diff_true = y_true[1:] - y_true[:-1]

    # Define Ψ(pred) and Ψ(true)
    psi_pred = tf.math.sigmoid(y_pred)
    psi_true = tf.math.sigmoid(y_true)

    # Calculate first-order difference of Ψ(pred) and Ψ(true)
    psi_diff_pred = psi_pred[1:] - psi_pred[:-1]
    psi_diff_true = psi_true[1:] - psi_true[:-1]

    # Calculate L2
    term1 = (1 - gamma) * tf.reduce_mean(tf.square(psi_pred - psi_true))
    term2 = gamma * tf.reduce_mean(tf.square(psi_diff_pred - psi_diff_true))
    L2 = term1 + term2

    return L2/N

In [ ]:
x = tf.ones(100)
y = tf.zeros(100)
print(L2_Loss(x,y))

tf.Tensor(0.0040041036, shape=(), dtype=float32)


In [ ]:
model = unet_baseline()
model.compile(loss=root_mean_squared_error, optimizer='adam')
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 32, 64, 1)]  0           []                               
                                                                                                  
 conv2d_21 (Conv2D)             (None, 32, 64, 32)   832         ['input_3[0][0]']                
                                                                                                  
 conv2d_22 (Conv2D)             (None, 32, 64, 32)   25632       ['conv2d_21[0][0]']              
                                                                                                  
 max_pooling2d_5 (MaxPooling2D)  (None, 16, 32, 32)  0           ['conv2d_22[0][0]']              
                                                                                            

In [ ]:
model = unet_old()
model.compile(loss=L2_Loss, optimizer='adam')
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 32, 64, 1)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 32, 64, 32)   320         ['input_1[0][0]']                
                                                                                                  
 max_pooling2d (MaxPooling2D)   (None, 16, 32, 32)   0           ['conv2d[0][0]']                 
                                                                                                  
 conv2d_1 (Conv2D)              (None, 16, 32, 32)   9248        ['max_pooling2d[0][0]']          
                                                                                              

Here, batch_size, and num_epochs have undergone HPO. Change variable "lead" for different "x" in U-STNx. Refer to [paper](https://gmd.copernicus.org/preprints/gmd-2021-71/) for details. For each value of lead a different weights file (including biases) would be saved. 

### Train Loop

In [ ]:
#Z = 8760, trainN = 8460
batch_size = 10
num_epochs = 8
lead=12
count=0
for loop in fileList_train:
    print('******************** counter ******************** COUNT =',count)
    File=nc.Dataset(loop)
    Z=np.asarray(File['z'])
    trainN=np.size(Z,0)-300
    Z=(Z-M)/sdev
    
    x_train=Z[0:trainN,:,:]
    x_train=x_train.reshape([np.size(x_train,0),32,64,1])
    y_train=Z[lead:trainN+lead,:,:]
    y_train=y_train.reshape([np.size(y_train,0),32,64,1])
    
    x_val= Z[trainN+lead:np.size(Z,0)-lead,:,:]
    x_val=x_val.reshape([np.size(x_val,0),32,64,1])
    
    y_val= Z[trainN+lead*2:np.size(Z,0),:,:]
    y_val=y_val.reshape([np.size(y_val,0),32,64,1])


    if (count>0):
        model = unet_baseline()
        model.compile(loss=root_mean_squared_error, optimizer='adam')
        model.load_weights('/content/drive/MyDrive/DDWP/persistence_rmse_2.h5')
        hist = model.fit(x_train, y_train,
                       batch_size = batch_size,
             verbose=1,
             epochs = 6,
             validation_data=(x_val,y_val),shuffle=True,
             callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss',
                                        min_delta=0,
                                        patience=5, # just to make sure we use a lot of patience before stopping
                                        verbose=0, mode='auto'),
                       keras.callbacks.ModelCheckpoint('/content/drive/MyDrive/DDWP/persistence_rmse_2.h5', monitor='val_loss',
                                                    verbose=1, save_best_only=True,
                                                    save_weights_only=True, mode='auto', period=1),history]
             )

    else:
        hist = model.fit(x_train, y_train,
                       batch_size = batch_size,
             verbose=1,
             epochs = 6,
             validation_data=(x_val,y_val),shuffle=True,
             callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss',
                                        min_delta=0,
                                        patience=5, # just to make sure we use a lot of patience before stopping
                                        verbose=0, mode='auto'),
                       keras.callbacks.ModelCheckpoint('/content/drive/MyDrive/DDWP/persistence_rmse_2.h5', monitor='val_loss',
                                                    verbose=1, save_best_only=True,
                                                    save_weights_only=True, mode='auto', period=1),history]
             )
    count=count+1

print("\nTraining Complete!")

******************** counter ******************** COUNT = 0


Epoch 1/6
846/846 [==============================] - ETA: 0s - loss: 0.1083
Epoch 1: val_loss improved from inf to 0.07922, saving model to /content/drive/MyDrive/DDWP/persistence_rmse_2.h5
846/846 [==============================] - 30s 11ms/step - loss: 0.1083 - val_loss: 0.0792
Epoch 2/6
841/846 [============================>.] - ETA: 0s - loss: 0.0691
Epoch 2: val_loss improved from 0.07922 to 0.07579, saving model to /content/drive/MyDrive/DDWP/persistence_rmse_2.h5
846/846 [==============================] - 9s 10ms/step - loss: 0.0691 - val_loss: 0.0758
Epoch 3/6
846/846 [==============================] - ETA: 0s - loss: 0.0595
Epoch 3: val_loss improved from 0.07579 to 0.07410, saving model to /content/drive/MyDrive/DDWP/persistence_rmse_2.h5
846/846 [==============================] - 10s 11ms/step - loss: 0.0595 - val_loss: 0.0741
Epoch 4/6
842/846 [============================>.] - ETA: 0s - loss: 0.0537
Epoch 4: val_loss improved from 0.07410 to 0.07263, saving model to /conte

Epoch 1/6
849/849 [==============================] - ETA: 0s - loss: 0.0609
Epoch 1: val_loss improved from inf to 0.06801, saving model to /content/drive/MyDrive/DDWP/persistence_rmse_2.h5
849/849 [==============================] - 16s 11ms/step - loss: 0.0609 - val_loss: 0.0680
Epoch 2/6
847/849 [============================>.] - ETA: 0s - loss: 0.0495
Epoch 2: val_loss improved from 0.06801 to 0.06663, saving model to /content/drive/MyDrive/DDWP/persistence_rmse_2.h5
849/849 [==============================] - 12s 14ms/step - loss: 0.0495 - val_loss: 0.0666
Epoch 3/6
848/849 [============================>.] - ETA: 0s - loss: 0.0459
Epoch 3: val_loss improved from 0.06663 to 0.06551, saving model to /content/drive/MyDrive/DDWP/persistence_rmse_2.h5
849/849 [==============================] - 10s 12ms/step - loss: 0.0459 - val_loss: 0.0655
Epoch 4/6
849/849 [==============================] - ETA: 0s - loss: 0.0437
Epoch 4: val_loss improved from 0.06551 to 0.06464, saving model to /cont

Epoch 1/6
843/846 [============================>.] - ETA: 0s - loss: 0.0565
Epoch 1: val_loss improved from inf to 0.06492, saving model to /content/drive/MyDrive/DDWP/persistence_rmse_2.h5
846/846 [==============================] - 15s 10ms/step - loss: 0.0565 - val_loss: 0.0649
Epoch 2/6
842/846 [============================>.] - ETA: 0s - loss: 0.0469
Epoch 2: val_loss did not improve from 0.06492
846/846 [==============================] - 9s 10ms/step - loss: 0.0469 - val_loss: 0.0674
Epoch 3/6
846/846 [==============================] - ETA: 0s - loss: 0.0437
Epoch 3: val_loss did not improve from 0.06492
846/846 [==============================] - 8s 10ms/step - loss: 0.0437 - val_loss: 0.0655
Epoch 4/6
846/846 [==============================] - ETA: 0s - loss: 0.0417
Epoch 4: val_loss did not improve from 0.06492
846/846 [==============================] - 9s 10ms/step - loss: 0.0417 - val_loss: 0.0719
Epoch 5/6
845/846 [============================>.] - ETA: 0s - loss: 0.0404
Epoc

Epoch 1/6
842/846 [============================>.] - ETA: 0s - loss: 0.0550
Epoch 1: val_loss improved from inf to 0.06106, saving model to /content/drive/MyDrive/DDWP/persistence_rmse_2.h5
846/846 [==============================] - 15s 10ms/step - loss: 0.0550 - val_loss: 0.0611
Epoch 2/6
845/846 [============================>.] - ETA: 0s - loss: 0.0456
Epoch 2: val_loss improved from 0.06106 to 0.05953, saving model to /content/drive/MyDrive/DDWP/persistence_rmse_2.h5
846/846 [==============================] - 9s 11ms/step - loss: 0.0456 - val_loss: 0.0595
Epoch 3/6
846/846 [==============================] - ETA: 0s - loss: 0.0424
Epoch 3: val_loss did not improve from 0.05953
846/846 [==============================] - 8s 10ms/step - loss: 0.0424 - val_loss: 0.0658
Epoch 4/6
844/846 [============================>.] - ETA: 0s - loss: 0.0406
Epoch 4: val_loss did not improve from 0.05953
846/846 [==============================] - 8s 10ms/step - loss: 0.0406 - val_loss: 0.0629
Epoch 5/6

...

Epoch 1/6
841/846 [============================>.] - ETA: 0s - loss: 0.0482
Epoch 1: val_loss improved from inf to 0.05674, saving model to /content/drive/MyDrive/DDWP/persistence_rmse_2.h5
846/846 [==============================] - 14s 11ms/step - loss: 0.0482 - val_loss: 0.0567
Epoch 2/6
843/846 [============================>.] - ETA: 0s - loss: 0.0412
Epoch 2: val_loss did not improve from 0.05674
846/846 [==============================] - 8s 9ms/step - loss: 0.0412 - val_loss: 0.0573
Epoch 3/6
846/846 [==============================] - ETA: 0s - loss: 0.0388
Epoch 3: val_loss did not improve from 0.05674
846/846 [==============================] - 9s 10ms/step - loss: 0.0388 - val_loss: 0.0583
Epoch 4/6
843/846 [============================>.] - ETA: 0s - loss: 0.0372
Epoch 4: val_loss did not improve from 0.05674
846/846 [==============================] - 8s 10ms/step - loss: 0.0372 - val_loss: 0.0587
Epoch 5/6
843/846 [============================>.] - ETA: 0s - loss: 0.0358
Epoch

Epoch 1/6
847/849 [============================>.] - ETA: 0s - loss: 0.0493
Epoch 1: val_loss improved from inf to 0.05992, saving model to /content/drive/MyDrive/DDWP/persistence_rmse_2.h5
849/849 [==============================] - 15s 11ms/step - loss: 0.0493 - val_loss: 0.0599
Epoch 2/6
846/849 [============================>.] - ETA: 0s - loss: 0.0419
Epoch 2: val_loss did not improve from 0.05992
849/849 [==============================] - 8s 10ms/step - loss: 0.0419 - val_loss: 0.0614
Epoch 3/6
848/849 [============================>.] - ETA: 0s - loss: 0.0393
Epoch 3: val_loss did not improve from 0.05992
849/849 [==============================] - 9s 10ms/step - loss: 0.0393 - val_loss: 0.0606
Epoch 4/6
849/849 [==============================] - ETA: 0s - loss: 0.0375
Epoch 4: val_loss did not improve from 0.05992
849/849 [==============================] - 9s 11ms/step - loss: 0.0375 - val_loss: 0.0612
Epoch 5/6
844/849 [============================>.] - ETA: 0s - loss: 0.0362
Epoc

Epoch 1/6
845/846 [============================>.] - ETA: 0s - loss: 0.0492
Epoch 1: val_loss improved from inf to 0.05259, saving model to /content/drive/MyDrive/DDWP/persistence_rmse_2.h5
846/846 [==============================] - 15s 11ms/step - loss: 0.0492 - val_loss: 0.0526
Epoch 2/6
844/846 [============================>.] - ETA: 0s - loss: 0.0419
Epoch 2: val_loss did not improve from 0.05259
846/846 [==============================] - 9s 10ms/step - loss: 0.0419 - val_loss: 0.0537
Epoch 3/6
842/846 [============================>.] - ETA: 0s - loss: 0.0395
Epoch 3: val_loss did not improve from 0.05259
846/846 [==============================] - 8s 9ms/step - loss: 0.0395 - val_loss: 0.0542
Epoch 4/6
846/846 [==============================] - ETA: 0s - loss: 0.0376
Epoch 4: val_loss did not improve from 0.05259
846/846 [==============================] - 9s 11ms/step - loss: 0.0376 - val_loss: 0.0564
Epoch 5/6
845/846 [============================>.] - ETA: 0s - loss: 0.0367
Epoch

Epoch 1/6
843/846 [============================>.] - ETA: 0s - loss: 0.0476
Epoch 1: val_loss improved from inf to 0.05796, saving model to /content/drive/MyDrive/DDWP/persistence_rmse_2.h5
846/846 [==============================] - 14s 10ms/step - loss: 0.0476 - val_loss: 0.0580
Epoch 2/6
844/846 [============================>.] - ETA: 0s - loss: 0.0407
Epoch 2: val_loss did not improve from 0.05796
846/846 [==============================] - 9s 11ms/step - loss: 0.0407 - val_loss: 0.0593
Epoch 3/6
842/846 [============================>.] - ETA: 0s - loss: 0.0382
Epoch 3: val_loss did not improve from 0.05796
846/846 [==============================] - 8s 9ms/step - loss: 0.0382 - val_loss: 0.0600
Epoch 4/6
845/846 [============================>.] - ETA: 0s - loss: 0.0367
Epoch 4: val_loss did not improve from 0.05796
846/846 [==============================] - 9s 10ms/step - loss: 0.0367 - val_loss: 0.0590
Epoch 5/6
844/846 [============================>.] - ETA: 0s - loss: 0.0357
Epoch

Epoch 1/6
844/846 [============================>.] - ETA: 0s - loss: 0.0482
Epoch 1: val_loss improved from inf to 0.05486, saving model to /content/drive/MyDrive/DDWP/persistence_rmse_2.h5
846/846 [==============================] - 15s 10ms/step - loss: 0.0482 - val_loss: 0.0549
Epoch 2/6
842/846 [============================>.] - ETA: 0s - loss: 0.0412
Epoch 2: val_loss did not improve from 0.05486
846/846 [==============================] - 9s 10ms/step - loss: 0.0412 - val_loss: 0.0573
Epoch 3/6
846/846 [==============================] - ETA: 0s - loss: 0.0389
Epoch 3: val_loss improved from 0.05486 to 0.05458, saving model to /content/drive/MyDrive/DDWP/persistence_rmse_2.h5
846/846 [==============================] - 9s 11ms/step - loss: 0.0389 - val_loss: 0.0546
Epoch 4/6
842/846 [============================>.] - ETA: 0s - loss: 0.0371
Epoch 4: val_loss did not improve from 0.05458
846/846 [==============================] - 9s 10ms/step - loss: 0.0371 - val_loss: 0.0547
Epoch 5/6